In [56]:
import requests
from bs4 import BeautifulSoup
import psycopg2
import time
from urllib.parse import urljoin

In [57]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="ja52712",
    port="5432"
)
print("Postgresに接続できた")



Postgresに接続できた


In [58]:
cursor = conn.cursor()

In [59]:
create_table_sql = """
create table if not exists users(
    id serial primary key,
    title text not null,
    product_url text unique not null
);
"""


conn.commit()

In [60]:
cursor.execute(create_table_sql)
conn.commit()
print("usersテーブル作成完了")

usersテーブル作成完了


In [61]:
cursor.execute("select * from users;")
rows = cursor.fetchall()
print(rows)

[(1, 'Test Book', 'https://books.toscrape.com/catalogue/test-book_1/index.html')]


In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2
import time
from urllib.parse import urljoin

#スクレイピングするURLを取得して、itemsに空のリスト、pageに初期設定を施した。
url = "https://books.toscrape.com/catalogue/page-1.html"
items = []
page = 1

#while文で次ページがある限り反復処理を行い、httpリクエストを送って、最大10秒待ち、返ってきたステータスコードをresに代入する。
#ステータスコードが200番台以外なら例外を発生させて処理を中止。（raise_for_status）
#ステータスコードが200番台ならres.text（html文字列）を構文解析してsoupオブジェクトを生成する。
while url:
    print(f"ページ {page} を取得中: {url}")
    res = requests.get(url,timeout=10)
    res.raise_for_status()
    soup = BeautifulSoup(res.text,"html.parser")
#articleタグ内のproduct_podクラス要素を順に取得し、生成したcardオブジェクトでそれぞれのタグ内の属性を取得。
#urljoin関数でベースのurlとhref属性のurlを結合して絶対パスを生成する。
#タイトルと絶対パスをタプルにまとめてitemsリストに追加していく。
    for card in soup.select("article.product_pod"):
        title = card.h3.a["title"].strip()
        rel = card.h3.a["href"]
        abs_url = urljoin(url,rel)
        items.append((title,abs_url))
#soupオブジェクトのselect_oneメソッドを使って、liタグのnextクラス直下にあるaタグを最初に見つけた1件のみ取得して、next_linkに代入する。
    next_link = soup.select_one("li.next > a")
#if文でnext_linkに値が入ったら、ベースurlとnext_link内のhref属性を結合して絶対パスを作成して、urlに代入する。
#代入されたら、page変数のカウントを1増やし、0.5秒待ってから次の処理に移り、next_linkに値が入らなければ、breakで強制終了する。
    if next_link:
        url = urljoin(url,next_link["href"])
        page += 1
        time.sleep(0.5)
    else:
        break

print(f"\n取得完了：総ページ数{page},総アイテム数{len(items)}")

📖 ページ 1 を取得中: https://books.toscrape.com/catalogue/page-1.html
📖 ページ 2 を取得中: https://books.toscrape.com/catalogue/page-2.html
📖 ページ 3 を取得中: https://books.toscrape.com/catalogue/page-3.html
📖 ページ 4 を取得中: https://books.toscrape.com/catalogue/page-4.html
📖 ページ 5 を取得中: https://books.toscrape.com/catalogue/page-5.html
📖 ページ 6 を取得中: https://books.toscrape.com/catalogue/page-6.html
📖 ページ 7 を取得中: https://books.toscrape.com/catalogue/page-7.html
📖 ページ 8 を取得中: https://books.toscrape.com/catalogue/page-8.html
📖 ページ 9 を取得中: https://books.toscrape.com/catalogue/page-9.html
📖 ページ 10 を取得中: https://books.toscrape.com/catalogue/page-10.html
📖 ページ 11 を取得中: https://books.toscrape.com/catalogue/page-11.html
📖 ページ 12 を取得中: https://books.toscrape.com/catalogue/page-12.html
📖 ページ 13 を取得中: https://books.toscrape.com/catalogue/page-13.html


KeyboardInterrupt: 